In [ ]:
import scipy.io
import h5py
import pandas as pd
import numpy as np
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/rijkmuseum')

In [ ]:
#Read in files
Xtrainchunk = pd.read_csv('Xtrain.txt',delimiter = ',',header=None,chunksize=1000)
Xtrain = pd.concat(Xtrainchunk)
Xtestchunk = pd.read_csv('Xtest.txt',delimiter = ',',header=None,chunksize=1000)
Xtest = pd.concat(Xtestchunk)
Xvalchunk = pd.read_csv('Xval.txt',delimiter = ',',header=None,chunksize=1000)
Xval = pd.concat(Xvalchunk)
Xntrainchunk = pd.read_csv('Xntrain.txt',delimiter=',',header=None,chunksize=1000)
Xntrain = pd.concat(Xntrainchunk)
Xntestchunk = pd.read_csv('Xntest.txt',delimiter=',',header=None,chunksize=1000)
Xntest = pd.concat(Xntestchunk)
Xnvalchunk = pd.read_csv('Xnval.txt',delimiter=',',header=None,chunksize=1000)
Xnval = pd.concat(Xnvalchunk)
Namekeytrain = pd.read_csv('Namekeytrain.txt',delimiter=',',header=None)
Namekeytest = pd.read_csv('Namekeytest.txt',delimiter=',',header=None)
Namekeyval = pd.read_csv('Namekeyval.txt',delimiter=',',header=None)
Nameguide = pd.read_csv('rijkNamefile.txt',delimiter='\t',header=None)
